In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import acquire
import wrangle
from scipy import stats
import sklearn.preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from numpy import percentile

from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_regression

    Filter out data using SQL propertylandusetypeid = 261 and transactiondate 2017

In [2]:
df = acquire.get_zillow_data()

Reading from csv file...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52441 non-null  int64  
 1   bedroomcnt                    52441 non-null  float64
 2   bathroomcnt                   52441 non-null  float64
 3   calculatedfinishedsquarefeet  52359 non-null  float64
 4   poolcnt                       11096 non-null  float64
 5   pooltypeid2                   1071 non-null   float64
 6   pooltypeid10                  444 non-null    float64
 7   fireplacecnt                  7243 non-null   float64
 8   garagecarcnt                  18015 non-null  float64
 9   yearbuilt                     52325 non-null  float64
 10  lotsizesquarefeet             52072 non-null  float64
 11  latitude                      52441 non-null  float64
 12  longitude                     52441 non-null  float64
 13  r

    Verify there are no duplicates in df. Yay no duplicates

In [4]:
df.duplicated().sum()

0

    How many NaN values in df

In [5]:
df_prep = df.copy(deep=True)
df_prep.isnull().sum()

parcelid                            0
bedroomcnt                          0
bathroomcnt                         0
calculatedfinishedsquarefeet       82
poolcnt                         41345
pooltypeid2                     51370
pooltypeid10                    51997
fireplacecnt                    45198
garagecarcnt                    34426
yearbuilt                         116
lotsizesquarefeet                 369
latitude                            0
longitude                           0
regionidcounty                      0
regionidzip                        26
fips                                0
taxvaluedollarcnt                   1
transactiondate                     0
propertylandusetypeid               0
dtype: int64

In [6]:
df_prep.bathroomcnt.value_counts()

2.0     21893
3.0     10673
1.0      9568
2.5      3934
4.0      2227
3.5       918
1.5       841
5.0       803
4.5       687
6.0       322
5.5       224
0.0       121
7.0        88
8.0        53
6.5        47
7.5        16
9.0        13
10.0        5
11.0        3
8.5         3
18.0        1
13.0        1
Name: bathroomcnt, dtype: int64

In [7]:
df_prep.bedroomcnt.value_counts()

3.0     23359
4.0     15240
2.0      8340
5.0      3973
6.0       635
1.0       612
0.0       137
7.0       106
8.0        24
9.0         8
12.0        3
10.0        2
14.0        1
11.0        1
Name: bedroomcnt, dtype: int64

    Replace zero values in bath & bed with NaN

In [8]:
bed_bath_0_columns = ['bedroomcnt', 'bathroomcnt']
df_prep[bed_bath_0_columns] = df_prep[bed_bath_0_columns].replace(0, np.NaN)
df_prep = df_prep.reset_index(drop=True)

In [9]:
df_prep.isnull().sum()

parcelid                            0
bedroomcnt                        137
bathroomcnt                       121
calculatedfinishedsquarefeet       82
poolcnt                         41345
pooltypeid2                     51370
pooltypeid10                    51997
fireplacecnt                    45198
garagecarcnt                    34426
yearbuilt                         116
lotsizesquarefeet                 369
latitude                            0
longitude                           0
regionidcounty                      0
regionidzip                        26
fips                                0
taxvaluedollarcnt                   1
transactiondate                     0
propertylandusetypeid               0
dtype: int64

    Since all fire/pool/gareage that NaN are 0 we will replace NaN values with 0 for pool, fire, and garage

In [10]:
fire_garge_pool_0_columns = ['poolcnt','pooltypeid2','pooltypeid10', 'fireplacecnt','garagecarcnt']
df_prep[fire_garge_pool_0_columns] = df_prep[fire_garge_pool_0_columns].replace(np.NaN, 0)
df_prep = df_prep.reset_index(drop=True)


In [11]:
df_prep.isnull().sum()

parcelid                          0
bedroomcnt                      137
bathroomcnt                     121
calculatedfinishedsquarefeet     82
poolcnt                           0
pooltypeid2                       0
pooltypeid10                      0
fireplacecnt                      0
garagecarcnt                      0
yearbuilt                       116
lotsizesquarefeet               369
latitude                          0
longitude                         0
regionidcounty                    0
regionidzip                      26
fips                              0
taxvaluedollarcnt                 1
transactiondate                   0
propertylandusetypeid             0
dtype: int64

In [12]:
df_prep.shape

(52441, 19)

    Since there are only 558 unique rows that contain NaN values we are going to drop all for the sake of time

In [13]:
df_prep = df_prep.dropna(axis=0)
df_prep = df_prep.reset_index(drop=True)

In [14]:
df_prep.shape

(51883, 19)

In [15]:
df_prep.head(50)

,parcelid,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,poolcnt,pooltypeid2,pooltypeid10,fireplacecnt,garagecarcnt,yearbuilt,lotsizesquarefeet,latitude,longitude,regionidcounty,regionidzip,fips,taxvaluedollarcnt,transactiondate,propertylandusetypeid
0,14297519,4.0,3.5,3100.0,0.0,0.0,0.0,0.0,2.0,1998.0,4506.0,33634931.0,-117869207.0,1286.0,96978.0,6059.0,1023282.0,2017-01-01,261.0
1,17052889,2.0,1.0,1465.0,0.0,0.0,0.0,1.0,1.0,1967.0,12647.0,34449266.0,-119281531.0,2061.0,97099.0,6111.0,464000.0,2017-01-01,261.0
2,14186244,3.0,2.0,1243.0,1.0,0.0,0.0,0.0,2.0,1962.0,8432.0,33886168.0,-117823170.0,1286.0,97078.0,6059.0,564778.0,2017-01-01,261.0
3,12177905,4.0,3.0,2376.0,1.0,0.0,0.0,0.0,0.0,1970.0,13038.0,34245180.0,-118240722.0,3101.0,96330.0,6037.0,145143.0,2017-01-01,261.0
4,12095076,4.0,3.0,2962.0,1.0,0.0,0.0,0.0,0.0,1950.0,63000.0,34145202.0,-118179824.0,3101.0,96293.0,6037.0,773303.0,2017-01-01,261.0
5,12069064,2.0,1.0,738.0,0.0,0.0,0.0,0.0,0.0,1922.0,4214.0,34149214.0,-118239357.0,3101.0,96325.0,6037.0,218552.0,2017-01-01,261.0
6,12790562,4.0,3.0,3039.0,0.0,0.0,0.0,0.0,0.0,1970.0,20028.0,33960230.0,-118006914.0,3101.0,96173.0,6037.0,220583.0,2017-01-02,261.0
7,11104527,4.0,3.0,2540.0,1.0,0.0,0.0,0.0,0.0,1999.0,10384.0,34434971.0,-118464390.0,3101.0,96374.0,6037.0,430108.0,2017-01-02,261.0
8,13944538,3.0,2.5,1340.0,0.0,0.0,0.0,1.0,2.0,1980.0,1199.0,33771438.0,-117982834.0,1286.0,97051.0,6059.0,319668.0,2017-01-02,261.0
9,17110996,3.0,2.5,1371.0,0.0,0.0,0.0,0.0,2.0,2004.0,3445.0,34238229.0,-119170633.0,2061.0,97107.0,6111.0,198054.0,2017-01-02,261.0


In [16]:
df_prep.columns

Index(['parcelid', 'bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'poolcnt', 'pooltypeid2', 'pooltypeid10', 'fireplacecnt',
       'garagecarcnt', 'yearbuilt', 'lotsizesquarefeet', 'latitude',
       'longitude', 'regionidcounty', 'regionidzip', 'fips',
       'taxvaluedollarcnt', 'transactiondate', 'propertylandusetypeid'],
      dtype='object')

In [17]:
df_prep.columns

Index(['parcelid', 'bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'poolcnt', 'pooltypeid2', 'pooltypeid10', 'fireplacecnt',
       'garagecarcnt', 'yearbuilt', 'lotsizesquarefeet', 'latitude',
       'longitude', 'regionidcounty', 'regionidzip', 'fips',
       'taxvaluedollarcnt', 'transactiondate', 'propertylandusetypeid'],
      dtype='object')

In [18]:
df_prep = df_prep.drop(columns=['parcelid', 'propertylandusetypeid', 'transactiondate'])

In [19]:
df_prep.columns

Index(['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'poolcnt',
       'pooltypeid2', 'pooltypeid10', 'fireplacecnt', 'garagecarcnt',
       'yearbuilt', 'lotsizesquarefeet', 'latitude', 'longitude',
       'regionidcounty', 'regionidzip', 'fips', 'taxvaluedollarcnt'],
      dtype='object')

In [20]:
df_prep = df_prep.rename(columns={'bedroomcnt':'bedrooms', 'bathroomcnt':'bathrooms', 'calculatedfinishedsquarefeet':'sqft_living', 'poolcnt':'pools', 'pooltypeid2':'pool_hottub_or_spa', 'pooltypeid10':'hottub_or_spa', 'fireplacecnt':'fireplaces', 'garagecarcnt':'garage_cars', 'yearbuilt':'year_built', 'lotsizesquarefeet':'sqft_lot', 'latitude':'lat', 'longitude':'long', 'regionidcounty':'countyid', 'regionidzip':'zip', 'fips':'fips','taxvaluedollarcnt': 'taxvalue'})

In [21]:
df_prep.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'pools', 'pool_hottub_or_spa',
       'hottub_or_spa', 'fireplaces', 'garage_cars', 'year_built', 'sqft_lot',
       'lat', 'long', 'countyid', 'zip', 'fips', 'taxvalue'],
      dtype='object')

In [22]:
df_prep.head(10)

,bedrooms,bathrooms,sqft_living,pools,pool_hottub_or_spa,hottub_or_spa,fireplaces,garage_cars,year_built,sqft_lot,lat,long,countyid,zip,fips,taxvalue
0,4.0,3.5,3100.0,0.0,0.0,0.0,0.0,2.0,1998.0,4506.0,33634931.0,-117869207.0,1286.0,96978.0,6059.0,1023282.0
1,2.0,1.0,1465.0,0.0,0.0,0.0,1.0,1.0,1967.0,12647.0,34449266.0,-119281531.0,2061.0,97099.0,6111.0,464000.0
2,3.0,2.0,1243.0,1.0,0.0,0.0,0.0,2.0,1962.0,8432.0,33886168.0,-117823170.0,1286.0,97078.0,6059.0,564778.0
3,4.0,3.0,2376.0,1.0,0.0,0.0,0.0,0.0,1970.0,13038.0,34245180.0,-118240722.0,3101.0,96330.0,6037.0,145143.0
4,4.0,3.0,2962.0,1.0,0.0,0.0,0.0,0.0,1950.0,63000.0,34145202.0,-118179824.0,3101.0,96293.0,6037.0,773303.0
5,2.0,1.0,738.0,0.0,0.0,0.0,0.0,0.0,1922.0,4214.0,34149214.0,-118239357.0,3101.0,96325.0,6037.0,218552.0
6,4.0,3.0,3039.0,0.0,0.0,0.0,0.0,0.0,1970.0,20028.0,33960230.0,-118006914.0,3101.0,96173.0,6037.0,220583.0
7,4.0,3.0,2540.0,1.0,0.0,0.0,0.0,0.0,1999.0,10384.0,34434971.0,-118464390.0,3101.0,96374.0,6037.0,430108.0
8,3.0,2.5,1340.0,0.0,0.0,0.0,1.0,2.0,1980.0,1199.0,33771438.0,-117982834.0,1286.0,97051.0,6059.0,319668.0
9,3.0,2.5,1371.0,0.0,0.0,0.0,0.0,2.0,2004.0,3445.0,34238229.0,-119170633.0,2061.0,97107.0,6111.0,198054.0


In [23]:
eda = df_prep.copy(deep=True)

    EDA

    split data

In [24]:
train_and_validate, test = train_test_split(eda, random_state=123, test_size=.2)
train, validate = train_test_split(train_and_validate, random_state=123, test_size=.2)

In [25]:
def corrstatsgraphs3(df):
    """
    Description
    ----
    Outputs the general statistical description of the dataframe,
    outputs the correlation heatmap with target label, and outputs a distribution plot.
    
    Parameters
    ----
    df(DataFrame):
        The dataframe for which information will be displayed.
        
    Returns
    ----
    useful stats, correlation, and subplots
    
    """
       
    # Heatmap with min -1 to max 1 to all variables
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html
    corr = df.corr()
    plt.subplots(figsize=(22, 17))
    plt.title("Heatmap", fontsize = 'x-large')
    mask = np.triu(np.ones_like(corr, dtype=bool))
    cmap = sns.diverging_palette(230, 21, as_cmap=True)
    sns.heatmap(corr, annot=True, mask = mask, cmap=cmap, vmin=-1, vmax=1
    )
    # Correlation Heatmap with min -1 to max 1 in conjuction with pd.corr 
    plt.figure(figsize=(10, 8)) 
    plt.title("Heatmap", fontsize = 'x-large')
    sns.heatmap(df.corr()[['taxvalue']].sort_values(by='taxvalue', 
    ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG'
    )
    sns.pairplot(df[['bedrooms', 'bathrooms', 'sqft_living', 'pools', 'pool_hottub_or_spa',
       'hottub_or_spa', 'fireplaces', 'garage_cars', 'year_built', 'sqft_lot',
       'countyid', 'zip', 'fips', 'taxvalue']], corner=True, kind='reg', plot_kws={'line_kws':{'color':'red'}}
    )
    # Correlation Heatmap with min -1 to max 1 in conjuction with pd.corr
    plt.figure(figsize=(16,10))
    df.corr()['taxvalue'].sort_values(ascending=False).plot(kind='bar', figsize=(20,5), cmap='BrBG'
    )
    sns.jointplot(x="bedrooms", y="taxvalue", data=df,  kind='reg', height=5, line_kws={'color': 'red'}
    )
    sns.jointplot(x="bedrooms", y="taxvalue", data=df,  kind='kde', line_kws={'color': 'red'}
    )
      
    sns.jointplot(x="bathrooms", y="taxvalue", data=df, ratio=5, kind='reg', height=5, line_kws={'color': 'red'}
    )
    sns.jointplot(x="bathrooms", y="taxvalue", data=df,  kind='kde', line_kws={'color': 'red'})

In [26]:
# corrstatsgraphs3(train)

    preprocessing 

In [27]:
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.X_y(train, validate, test)

In [28]:
X_train.head()

,bedrooms,bathrooms,sqft_living,pools,pool_hottub_or_spa,hottub_or_spa,fireplaces,garage_cars,year_built,sqft_lot,lat,long,countyid,zip,fips
26147,4.0,5.0,3654.0,0.0,0.0,0.0,0.0,0.0,2001.0,2420.0,33875227.0,-118402379.0,3101.0,96109.0,6037.0
23083,3.0,2.0,1344.0,0.0,0.0,0.0,0.0,2.0,1966.0,6200.0,33715954.0,-117886754.0,1286.0,97004.0,6059.0
7241,3.0,2.0,1488.0,0.0,0.0,0.0,0.0,0.0,2005.0,3511.0,33998747.0,-118243899.0,3101.0,95992.0,6037.0
49426,2.0,3.0,1999.0,0.0,0.0,0.0,1.0,2.0,1972.0,1830.0,33723641.0,-118055880.0,1286.0,96967.0,6059.0
11164,3.0,2.0,1257.0,1.0,0.0,0.0,0.0,0.0,1987.0,8953.0,34499290.0,-118629639.0,3101.0,96398.0,6037.0


    Baseline w/ fecture selection, scaling, and outliers removal

In [30]:
predictions = pd.DataFrame({'actual': validate.taxvalue})

In [31]:
predictions.head()

,actual
41380,165413.0
12242,74552.0
1865,147163.0
18048,59596.0
8242,299104.0


In [34]:
lm = LinearRegression()
lm.fit(X_train, y_train)
predictions['simple_lm'] = lm.predict(X_validate)

In [35]:
predictions.head()

,actual,simple_lm
41380,165413.0,177234.858194
12242,74552.0,334816.991923
1865,147163.0,678677.002522
18048,59596.0,85825.747480
8242,299104.0,433951.694249
